### Cognitive Modeling (02458): Homework 1 - Part 3: Psychometric Functions

---

_By Sebastian Sbirna (s190553) and Aleksander Frese (s163859)_

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

### Psychometric function - parameter computation

In this experiment, we have a psychometric function shaped in the form of a Gaussian CDF. We are given the stimulus levels and number of yes responses for each level, from a total of 50 different trials performed for each stimulus in part.

In [2]:
total_trial_no = 50

stim_levels = [0.4, 0.9, 1.2, 1.7, 2.3]
yes_resp_no = [1, 6, 13, 32, 49]

We are asked to estimate the parameters of the psychometric function. Since this function is represented as a CDF, we may inverse the CDF function in order to retrieve the probability density function of our experiment.
Knowing this, and ___noting the stimulus values as x___, we can write the psychometric function which is a expresses the response of the observer as:

$$ \phi[P(yes|x)]^{-1} = \frac{x - \mu}{\sigma}$$

The two unknown parameters are $\mu$ and $\sigma$, however they can be computed if we bring the psychometric equation to a linear form:

$$ \phi[P(yes|x)]^{-1} = \frac{1}{\sigma}x + (-\frac{\mu}{\sigma})$$

The equation is now written in the form $y = ax + b$, with x (_stimulus_) being known and y (_inverse of probabilities of responder saying yes for the different stimulus trials_) being computable. Using this, we will compute $a = \frac{1}{\sigma}$ and $b = -\frac{\mu}{\sigma}$.

First, we will compute the probabilities $P(Yes|stimulus)$, and after that, we will compute the inverse CDF of these probabilities, so that we have the whole range of values for our _y_ in the linear equation.

In [5]:
p_yes = np.array(yes_resp_no) / total_trial_no
print(p_yes)

[0.02 0.12 0.26 0.64 0.98]


In [7]:
import scipy.stats as stats

Y = stats.norm.ppf(p_yes)
print(Y)

[-2.05374891 -1.17498679 -0.64334541  0.35845879  2.05374891]


Now, we will compute the parameters $\mu$ and $\sigma$, using the matrix $Y$ which contains the values for the inverse CDF of the probability of 'yes' responses by observers, and the matrix $X$ with two columns representing, first, a column of values for the original stimulus and also a column of ones for the computation of the intercept (_b_), since the intercept value is always constant across our model. We will want to compute the matrix $\beta$, where we understand that $ \beta= \left[ {\begin{array}{c} slope(a) \\ intercept(b) \\ \end{array} } \right]$.

To perform the computation of finding $\beta$ from the formula $Y = X * \beta$, we will use the pseudoinverse of the matrix $X$, i.e. $X^{-1}$. 

Therefore: $X^{-1} * Y (= X^{-1} * X * \beta) = \beta$

In [12]:
X = np.concatenate((np.array(stim_levels).reshape(5,1), np.ones((5, 1))), axis = 1)
print(X)

[[0.4 1. ]
 [0.9 1. ]
 [1.2 1. ]
 [1.7 1. ]
 [2.3 1. ]]


The pseudoinverse of the matrix $X$ may be obtained in the following way:

In [15]:
np.linalg.pinv(X)

array([[-0.42056075, -0.18691589, -0.04672897,  0.18691589,  0.46728972],
       [ 0.74672897,  0.44299065,  0.26074766, -0.04299065, -0.40747664]])

By multiplying the pseudoinverse of X ($X^{-1}$) with the matrix Y, we may obtain the matrix $\beta$ and the results for the slope $a$ and the intercept $b$.

In [18]:
a, b = np.dot(np.linalg.pinv(X), Y)
np.dot(np.linalg.pinv(X), Y)

array([ 2.14011014, -3.07411787])

If we now have found $a$, we have simultaneously found $\sigma$. 

To compute $\mu$, we just need to plug in $\sigma$ into the equation for $b$ (whose value has just been found from the $\beta$ matrix).

In [19]:
sigma = 1/a
print(sigma)

0.46726567019117615


In [21]:
mu = b * -sigma
print(str(mu))

1.4364297449222871


Through this thought process, we have computed the $\mu = 1.43$ and $\sigma = 0.46$ values of the psychometric function.

### Psychometric function - sensitivity ($d'$) computation

Now, a follow-up experiment decides to only use 2 values for initial stimulus: _x = 1_ and _x = 2_. The observer can also respond in only two methods: 'high' (_which has the same meaning as 'yes' previously_) or 'low'. We are asked to compute the sensitivity ($d'$) of the observer. 

The sensitivity of this observer refers to distance between the gaussian PDF of the stimulus of high intensity (centered around the mean $\mu_s = 2$) and the gaussian PDF of the stimulus of low intensity (centered around the mean $\mu_s = 1$).

This can be computed in two ways:
1. We can use the formula for the sensitivity $d'$ of the equal variance observer, namely: $d' = \phi^{-1}(P(Hit)) - \phi^{-1}(P(FA))$. 

This can be computed using the values of __P(Hit)__ and __P(FA)__ from the psychometric function. 

Even though we do not have actual data regarding the number of 'high' responses in the experiment, we can compute that using our linear equation and the values for $\mu$ and $\sigma$, computed just earlier.
What you would get from the equation would be $$\phi^{-1}(P(High|x = 2)) = \phi^{-1}(P(Hit))$$ and $$\phi^{-1}(P(High|x = 1)) = \phi^{-1}(P(FA))$$, when plugging in x = 1 and x = 2, respectively, into the linear function.

There is, however, a more simple method.

2. We can standardize the difference between the two stimuli (_since the stimulus value corresponds to the mean of its corresponding PDF_) by the standard deviation of the psychometric function. The std $\sigma$ has just been computed above as a parameter to our model.

Therefore: $$d' = \frac{2 - 1}{\sigma}$$

In [37]:
d_prime = (2 - 1)/sigma
print(d_prime)

2.1401101424610585


In order to show that this computation is exactly equivalent to the longer method described above, let us compute $\phi^{-1}(P(High|x = 2))$ and $\phi^{-1}(P(High|x = 1))$ using the linear equation from above.

In [39]:
x_stimulus_high = 2
inverse_p_hit = (1/sigma) * x_stimulus_high + (-mu/sigma)
print(inverse_p_hit)

1.206102418881179


In [40]:
x_stimulus_low = 1
inverse_p_fa = (1/sigma) * x_stimulus_low + (-mu/sigma)
print(inverse_p_fa)

-0.9340077235798794


In [42]:
d_prime = inverse_p_hit - inverse_p_fa
print(d_prime)

2.1401101424610585


As we can see from our computations, the two methods of computing $d'$ are equivalent and yield the same result.